In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import copy
import random
from src.util import *
from src.algorithm import *
from src.transport_network import *
from sanity_test.graph_test import *
from sanity_test.path_test import *

## Generating Test Paths

We will define a function `generate_paths_for_tests` to generate test paths. This function includes random tests during peak times and boundary tests in the Lausanne area. We will use a seed to ensure reproducibility.

1. **Peak Time Random Tests**
- Randomly sample pairs of stops and generate paths for them during peak travel times (e.g., 10:00 - 12:00).

2. **Boundary Time Tests in Lausanne Area**
- Test paths between stops within the Lausanne area during late night and early morning hours (e.g., 23:00 - 01:00 and 05:30 - 07:30).

Additionally, `graph_sanity_check` is applied every time a new graph is built to perform the following sanity checks on the graph:
1. **No Zero Cycles**: Ensure there are no cycles with zero total weight.
2. **Edge Weight >= 0**: All edges should have non-negative weights (travel time).
3. **Walking Weight > 0**: Ensure that walking weights are positive.

In [2]:
def generate_paths_for_tests(seed=42):
    if seed is not None:
        random.seed(seed)

    sbb_network = TransportNetwork('data/sbb_timetable_stop_times.parquet', 'data/stop_to_stop.csv', 'data/stops.csv')
    stops = sbb_network.stops
    stop_to_stop = sbb_network.stop_to_stop
    id_to_stop = stops.set_index('stop_id')['stop_name'].to_dict()
    stop_to_id = stops.set_index('stop_name')['stop_id'].to_dict()

    # Peak hours for random test
    peak_time = ('10:00', '12:00')  
    
    # late night and early morning for boundary test
    boundary_times = [('23:00', '01:00'), ('05:30', '07:30')] 

    # Sample some random pairs of stops 
    sample_size = 10
    sampled_pairs = random.sample(list(stop_to_id.keys()), sample_size * 2)  # Get 2 * sample_size to form pairs
    sampled_pairs = [(sampled_pairs[i], sampled_pairs[i + 1]) for i in range(0, len(sampled_pairs), 2)]

    # Only do the boundary test in the Lausanne area
    lausanne_stops = [stop for stop in stop_to_id.keys() if 'Lausanne' in stop]
    lausanne_sample = random.sample(lausanne_stops, sample_size * 2)
    lausanne_pairs = [
        (lausanne_sample[i], lausanne_sample[i + 1]) 
        for i in range(0, len(lausanne_sample), 2)
    ]

    all_generated_paths = []

    # Peak time random test
    G_peak = sbb_network.build_graph(peak_time[0], peak_time[1])
    print(f'-----------Peak time test during {peak_time}-----------')
    graph_sanity_check(G_peak)
    print('...')
    for departure, destination in sampled_pairs:
        departure_id = stop_to_id.get(departure, None)
        destination_id = stop_to_id.get(destination, None)
        print(f"Testing path from {departure} (ID: {departure_id}) to {destination} (ID: {destination_id})")
        if not departure_id or not destination_id:
            print(f" Invalid departure or destination for test: {departure}, {destination}")
            continue
        try:
            paths = yen_ksp(G_peak, peak_time[0], departure_id, destination_id)
            if not paths:
                print(f" No paths found for {departure} to {destination} from {peak_time[0]} to {peak_time[1]}")
                continue
            all_generated_paths.extend(paths)
            print(f" Generated {len(paths)} paths for {departure} to {destination}")
        except KeyError as e:
            print(f" KeyError generating paths for {departure} to {destination}: {e}")
        except Exception as e:
            print(f" Error generating paths for {departure} to {destination}: {e}")

    # City center boundary test (contains 'Lausanne')
    for boundary_time in boundary_times:
        G_boundary = sbb_network.build_graph(boundary_time[0], boundary_time[1])
        print('\n')
        print(f'-----------Boundary time test during {boundary_time}-----------')
        graph_sanity_check(G_boundary)
        print('...')
        for departure, destination in lausanne_pairs:
            departure_id = stop_to_id.get(departure, None)
            destination_id = stop_to_id.get(destination, None)
            print(f"Testing path from {departure} (ID: {departure_id}) to {destination} (ID: {destination_id})")
            if not departure_id or not destination_id:
                print(f" Invalid departure or destination for test: {departure}, {destination}")
                continue
            try:
                paths = yen_ksp(G_boundary, boundary_time[0], departure_id, destination_id)
                if not paths:
                    print(f" No paths found for {departure} to {destination} from {boundary_time[0]} to {boundary_time[1]}")
                    continue
                all_generated_paths.extend(paths)
                print(f" Generated {len(paths)} paths for {departure} to {destination}")
            except KeyError as e:
                print(f" KeyError generating paths for {departure} to {destination}: {e}")
            except Exception as e:
                print(f" Error generating paths for {departure} to {destination}: {e}")

    return all_generated_paths


In [3]:
generated_paths = generate_paths_for_tests()

-----------Peak time test during ('10:00', '12:00')-----------
Graph with 504 nodes passed the sanity check!
...
Testing path from Lausanne, Bellerive (ID: 8591989) to Epalinges, Grand Chemin (ID: 8591968)
 Generated 5 paths for Lausanne, Bellerive to Epalinges, Grand Chemin
Testing path from Bussigny, Tatironne (ID: 8506905) to Lausanne, Praz-Berthoud (ID: 8592099)
 Generated 5 paths for Bussigny, Tatironne to Lausanne, Praz-Berthoud
Testing path from Lausanne, Riant-Cour (ID: 8592110) to Lausanne, Ours (ID: 8579237)
 Generated 5 paths for Lausanne, Riant-Cour to Lausanne, Ours
Testing path from Lausanne, Maladière-Figuiers (ID: 8593837) to Lausanne, Blécherette (ID: 8591994)
 Generated 5 paths for Lausanne, Maladière-Figuiers to Lausanne, Blécherette
Testing path from Lausanne, Petit Château (ID: 8592091) to Ecublens VD, Perrettes (ID: 8591954)
 Generated 5 paths for Lausanne, Petit Château to Ecublens VD, Perrettes
Testing path from Lausanne, EPSIC (ID: 8592041) to Pully, port (ID: 

## Path Sanity Check

We perform the following `path_sanity_check` on the generated paths:

1. **Departure Time >= Arrival Time of the Previous Stop**: Ensure that the departure time from each stop is not earlier than the arrival time at the previous stop.
2. **Transfer Time >= 2 Minutes**: Ensure that the departure time at any transfer stop is at least 2 minutes later than the last arrival time.
3. **Walking Time <= 10 Minutes**: Ensure that walking time between stops does not exceed 10 minutes.

These checks ensure that the generated paths are realistic and adhere to our assumptions.

In [4]:
path_sanity_check(generated_paths)

All paths passed the sanity check.


## Compare with Google Maps:
Some examples

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import copy
import random
from src.util import *
from src.algorithm import *
from src.transport_network import *
from sanity_test.graph_test import *
from sanity_test.path_test import *

### 1. Lausanne, Bellerive -> Ecublens, EPFL:
- Arrive by 08:10 AM, Monday 27th May

In [110]:
start_time = '07:40'
expected_time = "08:10"
departure = "Lausanne, Bellerive"
destination = "Ecublens VD, EPFL"

In [111]:
sbb_network = TransportNetwork('data/sbb_timetable_stop_times.parquet', 'data/stop_to_stop.csv', 'data/stops.csv')
stops = sbb_network.stops
stop_info = stops.set_index('stop_id')[['stop_lat', 'stop_lon', 'stop_name']].to_dict('index')
stop_to_stop = sbb_network.stop_to_stop
id_to_stop = stops.set_index('stop_id')['stop_name'].to_dict()
stop_to_id = stops.set_index('stop_name')['stop_id'].to_dict()
G = sbb_network.build_graph(start_time, expected_time)
departure_id = stop_to_id[departure]
destination_id = stop_to_id[destination]

In [112]:
import importlib
import src.algorithm
importlib.reload(src.algorithm)
from src.algorithm import *

In [113]:
paths = yen_ksp(G, start_time, departure_id, destination_id)
print_paths(paths, id_to_stop, stop_info, limit = 3)

Path 1: Transfers: 1, Cost: 24 minutes, Walking: 0 minutes
Depart from Lausanne, Bellerive(8591989) at 7:45: 
            Lausanne, Théâtre de Vidy(8592121) at 7:46 via 24
            Lausanne, Vidy-Port(8595938) at 7:48 via 24
            Lausanne, Stade de Coubertin(8595935) at 7:49 via 24
            Lausanne, Siège du CIO(8595934) at 7:51 via 24
            Lausanne, Parc Bourget(8595939) at 7:52 via 24
            Lausanne, sports UNIL-EPFL(8595937) at 7:55 via 24
            Lausanne, Bourdonnette(8501210) at 7:57 via 24
                          (8501210-transfer) at 7:59 via transfer
            Chavannes-R., UNIL-Chamberonne(8501211) at 8:04 via m1
            Chavannes-R., UNIL-Mouline(8501212) at 8:05 via m1
            Ecublens VD, UNIL-Sorge(8501213) at 8:06 via m1
            Ecublens VD, EPFL(8501214) at 8:08 via m1
Arrival at Ecublens VD, EPFL(8501214) at 8:08
Path 2: Transfers: 1, Cost: 24 minutes, Walking: 7 minutes
Depart from Lausanne, Bellerive(8591989) at 7:45: 
 

**The planner shows similar results as google map**
![Google Map Route](img/google_epfl.png)

### 2. Lausanne, Bellerive -> Lausanne, Gare:
- Arrive by 6:00 AM, Monday 27th May

In [100]:
start_time = '05:40'
expected_time = "06:00"
departure = "Lausanne, Bellerive"
destination = "Lausanne, gare"
sbb_network = TransportNetwork('data/sbb_timetable_stop_times.parquet', 'data/stop_to_stop.csv', 'data/stops.csv')
stops = sbb_network.stops
stop_info = stops.set_index('stop_id')[['stop_lat', 'stop_lon', 'stop_name']].to_dict('index')
stop_to_stop = sbb_network.stop_to_stop
id_to_stop = stops.set_index('stop_id')['stop_name'].to_dict()
stop_to_id = stops.set_index('stop_name')['stop_id'].to_dict()
G = sbb_network.build_graph(start_time, expected_time)
departure_id = stop_to_id[departure]
destination_id = stop_to_id[destination]

In [101]:
paths = yen_ksp(G, start_time, departure_id, destination_id)
print_paths(paths, id_to_stop, stop_info, limit = 3)

Path 1: Transfers: 1, Cost: 11 minutes, Walking: 0 minutes
Depart from Lausanne, Bellerive(8591989) at 5:42: 
            Lausanne, Pêcheurs(8592090) at 5:42 via 2
            Lausanne, Ouchy-Olympique(8592086) at 5:44 via 2
                          (8592086-transfer) at 5:46 via transfer
            Lausanne, Jordils(8592061) at 5:50 via m2
            Lausanne, Délices(8592028) at 5:51 via m2
                          (8592052) at 5:52 via m2
            Lausanne, gare(8592050) at 5:53 via m2
Arrival at Lausanne, gare(8592050) at 5:53
Path 2: Transfers: 1, Cost: 11 minutes, Walking: 0 minutes
Depart from Lausanne, Bellerive(8591989) at 5:42: 
            Lausanne, Pêcheurs(8592090) at 5:42 via 2
            Lausanne, Ouchy-Olympique(8592086) at 5:44 via 2
            Lausanne, Beau-Rivage(8591986) at 5:45 via 2
            Lausanne, Jordils(8592061) at 5:46 via 2
                          (8592061-transfer) at 5:48 via transfer
            Lausanne, Délices(8592028) at 5:51 via m2
 

**The planner shows similar results as google map**
![Google Map Route](img/google_gare.png)

### 3. Lutry, Colline -> Renens VD, Censuy:
- Arrive by 19:20 PM, Monday 27th May

In [108]:
start_time = '18:40'
expected_time = "19:20"
departure = "Lutry, Colline"
destination = "Renens VD, Censuy"
sbb_network = TransportNetwork('data/sbb_timetable_stop_times.parquet', 'data/stop_to_stop.csv', 'data/stops.csv')
stops = sbb_network.stops
stop_info = stops.set_index('stop_id')[['stop_lat', 'stop_lon', 'stop_name']].to_dict('index')
stop_to_stop = sbb_network.stop_to_stop
id_to_stop = stops.set_index('stop_id')['stop_name'].to_dict()
stop_to_id = stops.set_index('stop_name')['stop_id'].to_dict()
G = sbb_network.build_graph(start_time, expected_time)
departure_id = stop_to_id[departure]
destination_id = stop_to_id[destination]

In [109]:
paths = yen_ksp(G, start_time, departure_id, destination_id)
print_paths(paths, id_to_stop, stop_info, limit = 3)

Path 1: Transfers: 2, Cost: 29 minutes, Walking: 5 minutes
Depart from Lutry, Colline(8592151) at 18:43: 
            La Conversion, gare(8591976) at 18:44 via 68
            La Conversion(8504010) at 18:46 via walking
                          (8504010-transfer) at 18:48 via transfer
            Pully-Nord(8504000) at 18:51 via 6
            Lausanne(8501120) at 18:55 via 6
            Prilly-Malley(8518452) at 19:01 via 6
            Renens VD(8501118) at 19:04 via 6
            Renens VD, gare sud(8588983) at 19:07 via walking
                          (8588983-transfer) at 19:09 via transfer
            Renens VD, Censuy(8592209) at 19:12 via 25
Arrival at Renens VD, Censuy(8592209) at 19:12
Path 2: Transfers: 2, Cost: 29 minutes, Walking: 10 minutes
Depart from Lutry, Colline(8592151) at 18:40: 
            La Conversion(8504010) at 18:47 via walking
                          (8504010-transfer) at 18:49 via transfer
            Pully-Nord(8504000) at 18:51 via 6
            Lausan

**The planner shows similar results as google map**
![Google Map Route](img/google_censuy.png)